In [30]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import torch
from sklearn.metrics import precision_recall_fscore_support
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from dino_experiments.util import get_embeddings, get_seeded_data_loader, Logger, AVAILABLE_DATASETS
GLOBAL_SEED = 42
RESIZE = True
BATCH_SIZE = 16
TEST_SAMPLE_SIZE = VAL_SAMPLE_SIZE = TRAIN_SAMPLE_SIZE = -1

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = 'fayoum'

In [3]:
test_loader = get_seeded_data_loader(dataset, "test", GLOBAL_SEED, resize=RESIZE, batch_size=BATCH_SIZE)
transformers = {'dino_vits8': torch.hub.load('facebookresearch/dino:main', 'dino_vits8', verbose=False)}
transformer = list(transformers.values())[0]

Splitting train and test data.
Splitting train and val data.


In [23]:
clf = MLPClassifier(random_state=GLOBAL_SEED, max_iter=10000)

In [5]:
X_test, y_test = get_embeddings(transformer, test_loader, TEST_SAMPLE_SIZE, batch_size=BATCH_SIZE)

In [22]:
train_loader = get_seeded_data_loader(dataset, "train", GLOBAL_SEED, batch_size=BATCH_SIZE, resize=RESIZE)
X_train, y_train = get_embeddings(transformer, train_loader, TRAIN_SAMPLE_SIZE, batch_size=BATCH_SIZE)

Splitting train and test data.
Splitting train and val data.


In [25]:
clf.fit(X_train, y_train)

MLPClassifier(max_iter=10000, random_state=42)

In [26]:
preds = clf.predict(X_test)

In [28]:
acc = clf.score(X_test, y_test)
acc

0.9444444444444444

In [31]:
prec, rec, f1, _ = precision_recall_fscore_support(y_test, preds)

In [32]:
prec, rec, f1, _

(array([0.96296296, 0.83333333, 0.94117647, 1.        ]),
 array([1.        , 0.71428571, 0.94117647, 1.        ]),
 array([0.98113208, 0.76923077, 0.94117647, 1.        ]),
 array([26,  7, 17,  4]))

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import torch
from sklearn.metrics import precision_recall_fscore_support
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from dino_experiments.util import get_embeddings, get_seeded_data_loader, Logger, AVAILABLE_DATASETS
GLOBAL_SEED = 42
RESIZE = True
BATCH_SIZE = 16
TEST_SAMPLE_SIZE = VAL_SAMPLE_SIZE = TRAIN_SAMPLE_SIZE = -1

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = 'fayoum'

test_loader = get_seeded_data_loader(dataset, "test", GLOBAL_SEED, resize=RESIZE, batch_size=BATCH_SIZE)
transformers = {'dino_vits8': torch.hub.load('facebookresearch/dino:main', 'dino_vits8', verbose=False)}
transformer = list(transformers.values())[0]

clf = MLPClassifier(random_state=GLOBAL_SEED, max_iter=10000)

X_test, y_test = get_embeddings(transformer, test_loader, TEST_SAMPLE_SIZE, batch_size=BATCH_SIZE)

train_loader = get_seeded_data_loader(dataset, "train", GLOBAL_SEED, batch_size=BATCH_SIZE, resize=RESIZE)
X_train, y_train = get_embeddings(transformer, train_loader, TRAIN_SAMPLE_SIZE, batch_size=BATCH_SIZE)

clf.fit(X_train, y_train)

preds = clf.predict(X_test)

acc = clf.score(X_test, y_test)
acc

prec, rec, f1, _ = precision_recall_fscore_support(y_test, preds)

prec, rec, f1, _